## ideas:

- sankey chart (catogrise spendings)
- detect subscriptions / recurring costs and list them

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import umap
from src.dashboard_utility import get_all_bank_data
import plotly.express as px

import plotly.io as pio
pio.templates.default = "plotly"

In [ ]:
comdirect_df, traderepublic_df, olb_df = get_all_bank_data()
df = comdirect_df.copy()